In [2]:
import FeedForward_Network as fn
import forward_propagation as fp
import backward_propagation as bp
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import time
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import pickle
import datetime

np.random.seed(6)

Using TensorFlow backend.


In [3]:
#save parameters
def save_pickle(a,filename):
    with open(filename+'.pickle', 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

#load parameters
def load_pickle(filename):
    with open(filename+'.pickle', 'rb') as handle:
        b = pickle.load(handle)
        return b

#save logs file
def writeToFile(stat, text, filename):
    if stat == 0:
        file = open('logs/'+filename+'_log_file.txt', "w")
        file.write("***** log file start *****" +str(datetime.datetime.now()) +"\n")
        file.write(text + "\n")
    if stat == 1:
        file = open('logs/'+filename+'_log_file.txt', "a")
        file.write(text + '\n')
    file.flush()
    file.close()

#list to string    
def textargs(*args):
    text=','.join([str(arg) for arg in args])
    return text

#batch size **2
def batch_list(x=64,maxb=1024):
    li=[]
    while x<= 1024:
        li.append(x)
        x *= 2        
    return li

In [11]:
# =============================================================================
#  load small data
# =============================================================================

In [6]:
data = pd.read_csv('data/seeds_norm.csv')
data=data.sample(frac=1)
x_train = data.iloc[:,:-1].values
y_train = data.iloc[:,-1].values

y_train=y_train-1

nb_classes = 3
targets = y_train.reshape(-1)
y_train = np.eye(nb_classes)[targets]
layers_dims = [7, 20, 10, 5 , 3]

In [8]:
parameters=None
for i in range(5):
    parameters,cost=fn.L_layer_model(x_train.T, y_train, layers_dims, 0.009, num_iterations = 100,batch_size=4,param=parameters,use_batchnorm=1,dropout_keep_prob=1)
    print(fn.predict(x_train,y_train,parameters))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
102.38177154127322
Accuracy: 0.7666666666666666
0.7666666666666666
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
96.19034312397424
Accuracy: 0.5904761904761904
0.5904761904761904
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


C:\Users\mor\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


In [10]:
# =============================================================================
# load mnist data
# =============================================================================

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float64')
x_test = x_test.astype('float64')

#y one hot
nb_classes = 10
targets = y_train.reshape(-1)
y_train = np.eye(nb_classes)[targets]

targets_test = y_test.reshape(-1)
y_test = np.eye(nb_classes)[targets_test]

#norm
x_train /= 255
x_test /= 255

#flat
height , weight = x_train.shape[1:]
image_vector_size= height * weight
x_train = x_train.reshape(x_train.shape[0], image_vector_size)
x_test = x_test.reshape(x_test.shape[0], image_vector_size)

#train validation
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

#layers of net
layers_dims = [784, 20, 7, 5, 10]

In [12]:
# =============================================================================
# test run for model
# =============================================================================

In [13]:
parameters=None
for i in range(10):
    print('\n')
    print('cost')
    parameters,cost=fn.L_layer_model(x_train.T, y_train, layers_dims, 0.009, num_iterations = 100,batch_size=1000,param=parameters,use_batchnorm=0,dropout_keep_prob=0.7)
    print('train')
    fn.predict(x_train,y_train,parameters)
    print('valid')
    fn.predict(x_valid,y_valid,parameters)



cost
0
1
2
3


KeyboardInterrupt: 

In [14]:
# =============================================================================
# running tests dropout and batch size
# =============================================================================

In [15]:
for drop_keep in np.arange(0.6,1,0.05):
    for batch_size in batch_list():
        
        acc_train_list=[]
        acc_valid_list=[]
        acc_test_list=[]
        costs=[]
        
        eps=1e-10
        acc_last=0
        num=1
        iterations=100
        epochs=0

        improve=True
        save_param=True
        parameters=None
    
        t=time.time()
        batch_drop='dr_'+str(round(drop_keep,2))+'_bs_'+str(batch_size)+'_'
        writeToFile(0,'acc_train,acc_valid,acc_test,cost,runtime,epochs,batch_size,timestamp',batch_drop)
        while improve:
        
            parameters,cost = fn.L_layer_model(x_train.T, y_train, layers_dims, 0.009, num_iterations = iterations,batch_size=batch_size,use_batchnorm=True,param=parameters,dropout_keep_prob=drop_keep)
            costs.append(cost[-1])
            
            acc_train=fn.predict(x_train,y_train,parameters)
            acc_valid = fn.predict(x_valid,y_valid,parameters)
            acc_test = fn.predict(x_test,y_test,parameters)
            
            if acc_last > acc_valid or acc_valid - acc_last < eps:
                improve=False
                save_param=False
            
            acc_last=acc_valid
            epochs=num*iterations
            runtime=time.time()-t
            writeToFile(1,textargs(acc_train,acc_valid,acc_test,cost[-1],runtime,epochs,batch_size,str(datetime.datetime.now())),batch_drop)
            
            if save_param:
                now = datetime.datetime.now()
                pick_time='models/ep'+str(epochs)+'__bs'+str(batch_size)+'__dr'+str(round(drop_keep,2))+'_'+str(now.strftime("%Y-%m-%d_%H_%M_%S"))
                save_pickle(parameters,pick_time)
            print('epochs: ',epochs)
            num+=1

0
1
2
3


KeyboardInterrupt: 